In [28]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockaing data:')
# print(df)
global_df

Global blockaing data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash,blocks_per_day,block_reward_btc,daily_emmisions_btc
BCH,518.0000,3.125000,3.405002e-03,SHA-256,3281605024601140926,166.795367,0.010641,1.774808
BTC,565.0000,3.137409,1.000000e+00,SHA-256,867866715045222654867,152.920354,3.137409,479.773629
DOGE,62.0000,10000.000000,2.573145e-06,Scrypt,2554841049724582,1393.548387,0.025731,35.858023
KAS,0.9998,65.460572,9.205924e-07,kHeavyHash,1375822395766898379,86417.283457,0.000060,5.207722
LTC,147.0000,6.250000,1.187800e-03,Scrypt,2228566262603385,587.755102,0.007424,4.363347
RVN,60.0000,2500.000000,1.434363e-07,KawPow,6607940577574,1440.000000,0.000359,0.516371


In [24]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
market_df = pd.DataFrame(index=index)
market_df['hashrate'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / factors_data_df['market_factor']
market_df['paying'] = algo_data_df['paying'] * factors_data_df['price_factor'] / sats2btc
market_df['daily_consumption_btc'] = market_df.hashrate * market_df.paying
print('Nicehash marketplace data:')
# print(market_data_df)
market_df

Nicehash marketplace data:


,hashrate,paying,daily_consumption_btc
SCRYPT,123.446906,0.016709,2.062640
SHA256ASICBOOST,6.986738,0.546248,3.816490
KAWPOW,0.788312,0.080875,0.063755
KHEAVYHASH,55.228909,0.003728,0.205894


In [25]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
market_df = pd.DataFrame(index=index)
market_df['hashrate'] = algo_data_df['speed'] * factors_data_df['mining_factor'] # H/s
market_df['paying'] = algo_data_df['paying'] / sats2btc # BTC/H/day
market_df['daily_consumption_btc'] = market_df.hashrate * market_df.paying
print('Nicehash marketplace data:')
# print(market_data_df)
market_df

Nicehash marketplace data:


,hashrate,paying,daily_consumption_btc
SCRYPT,1.234469e+14,1.670872e-14,2.062640
SHA256ASICBOOST,6.986738e+18,5.462477e-19,3.816490
KAWPOW,7.883122e+11,8.087513e-14,0.063755
KHEAVYHASH,5.522891e+16,3.728006e-18,0.205894


In [42]:
min_order_amount_btc = 5e-3
max_market_utilisation = 0.76

limits_df = pd.DataFrame(index=market_df.index)
limits_df['max_hashrate'] = market_df.hashrate * max_market_utilisation
limits_df['max_daily_consumption_btc'] = market_df.daily_consumption_btc * max_market_utilisation
limits_df['max_hourly_consumption_btc'] = limits_df.max_daily_consumption_btc / 24
limits_df['max_hourly_consumption_dollar'] = round(limits_df.max_hourly_consumption_btc * btc_exchange_rate, 2)
limits_df

,max_hashrate,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_dollar
SCRYPT,9.381965e+13,1.567606,0.065317,6390.70
SHA256ASICBOOST,5.309921e+18,2.900532,0.120856,11824.67
KAWPOW,5.991172e+11,0.048454,0.002019,197.53
KHEAVYHASH,4.197397e+16,0.156479,0.006520,637.92


In [49]:
reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['global_hashrate'] = global_df['nethash']
reachable_df.loc['BCH', 'nh_hashrate'] = limits_df.loc['SHA256ASICBOOST', 'max_hashrate']
reachable_df.loc['BTC', 'nh_hashrate'] = limits_df.loc['SHA256ASICBOOST', 'max_hashrate']
reachable_df.loc['DOGE', 'nh_hashrate'] = limits_df.loc['SCRYPT', 'max_hashrate']
reachable_df.loc['LTC', 'nh_hashrate'] = limits_df.loc['SCRYPT', 'max_hashrate']
reachable_df.loc['KAS', 'nh_hashrate'] = limits_df.loc['KHEAVYHASH', 'max_hashrate']
reachable_df.loc['RVN', 'nh_hashrate'] = limits_df.loc['KAWPOW', 'max_hashrate']
reachable_df
#limits_df.loc['SHA256ASICBOOST', ['max_hashrate']]

,algorithm,global_hashrate,nh_hashrate
BCH,SHA-256,3281605024601140926,5.309921e+18
BTC,SHA-256,867866715045222654867,5.309921e+18
DOGE,Scrypt,2554841049724582,9.381965e+13
KAS,kHeavyHash,1375822395766898379,4.197397e+16
LTC,Scrypt,2228566262603385,9.381965e+13
RVN,KawPow,6607940577574,5.991172e+11
